# Auto-Renaming My Untitled.ipynb Files

I'm starting to accumulate many UntitledX.ipynb files. Here I use an LLM to rename each one based on its contents.

In [23]:
from datetime import datetime
from execnb.nbio import read_nb
from fastcore.utils import *
import google.generativeai as genai
from pathlib import Path

## Desired File Format

YYYY-MM-DD-Title-of-Notebook-in-TitleCase-With-Hyphens.ipynb

## Get the Untitled Notebooks

In [56]:
nbs = L(Path().glob("Untitled*.ipynb"))
nbs

(#32) [Path('Untitled10.ipynb'),Path('Untitled7.ipynb'),Path('Untitled12.ipynb'),Path('Untitled5.ipynb'),Path('Untitled1.ipynb'),Path('Untitled16.ipynb'),Path('Untitled30.ipynb'),Path('Untitled29.ipynb'),Path('Untitled3.ipynb'),Path('Untitled14.ipynb'),Path('Untitled4.ipynb'),Path('Untitled13.ipynb'),Path('Untitled6.ipynb'),Path('Untitled11.ipynb'),Path('Untitled1-Copy1.ipynb'),Path('Untitled15.ipynb'),Path('Untitled2.ipynb'),Path('Untitled28.ipynb'),Path('Untitled17.ipynb'),Path('Untitled26.ipynb')...]

In [64]:
nb = nbs[0]

## Get the Last Modified Date

To get each file's prefix, I get the file modified stats:

In [65]:
Path(nb).stat().st_mtime

1736267381.7581108

This returns a Unix timestamp. To get a more readable datetime:

In [66]:
last_modified = datetime.fromtimestamp(Path(nb).stat().st_mtime)
last_modified

datetime.datetime(2025, 1, 7, 16, 29, 41, 758111)

In [67]:
last_modified.strftime('%Y-%m-%d')

'2025-01-07'

## Check for an Existing Title

It would be in the first cell:

In [68]:
cells = L(read_nb(nb).cells)
cells[0]

```json
{ 'cell_type': 'markdown',
  'id': 'c68e6923',
  'idx_': 0,
  'metadata': {},
  'source': 'git-nbdiffdriver diff: git-nbdiffdriver: command not found'}
```

## Ask Gemini to Title the Notebook

In [69]:
model = genai.GenerativeModel('gemini-1.5-flash-latest')
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash-latest',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [71]:
with open(x) as f:
    nb = f.read()

In [72]:
def generate_title_part(nb):
    prompt = f"""Given this Jupyter notebook, create a filename following these EXACT steps:
1. Extract the title from the first cell if it starts with '#'. In this case it's: "FastHTML By Example, Part 2"
2. Convert to the format: Words-In-Title-Case-With-Hyphens.ipynb
3. Remove any special characters (like commas)
4. If the filename sounds repetitive, simplify it.
5. If the first cell does not contain a title, create one based on the entire notebook's contents.

<notebook>
{nb}
</notebook>

Return ONLY the filename, nothing else."""
    safety_settings = [
        {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE",},
        {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE",},
        {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE",},
        {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE",},
    ]
    response = model.generate_content(prompt, safety_settings=safety_settings, request_options = {"timeout": 1000})
    try:
        return response.text
    except Exception as ex:
        raise ex

In [73]:
result = generate_title_part(nb)
print(result)

Git-Nbdiffdriver-And-Nbstripout-Issues.ipynb



## Prefix the Title With the Date

Putting the full title together:

In [74]:
full_title = f"{last_modified.strftime('%Y-%m-%d')}-{result.strip()}"
print(full_title)

2025-01-07-Git-Nbdiffdriver-And-Nbstripout-Issues.ipynb


## Rename the File

In [75]:
x

Path('Untitled10.ipynb')

In [77]:
new_path = Path(full_title)

In [78]:
if new_path.exists():
    print(f"Warning: {new_path} already exists")
else:
    x.rename(new_path)
    print(f"Renamed {x} to {new_path}")

Renamed Untitled10.ipynb to 2025-01-07-Git-Nbdiffdriver-And-Nbstripout-Issues.ipynb


## Make This All a Function

Let's put this all together into a function that we can call on several files.

In [90]:
def rename_notebook(nb_path):
    """Rename an untitled notebook based on its contents and modification date"""
    date = datetime.fromtimestamp(Path(nb_path).stat().st_mtime).strftime('%Y-%m-%d')
    with open(nb_path) as f: nb = f.read()
    
    title_part = generate_title_part(nb)
    
    new_name = f"{date}-{title_part.strip()}"
    new_path = Path(new_name)
    
    if new_path.exists():
        print(f"Warning: {new_path} already exists")
        return nb_path
    else:
        nb_path.rename(new_path)
        print(f"Renamed {nb_path} to {new_path}")
        return new_path

In [93]:
nbs = L(Path().glob("Untitled*.ipynb"))
nbs

(#30) [Path('Untitled12.ipynb'),Path('Untitled5.ipynb'),Path('Untitled1.ipynb'),Path('Untitled16.ipynb'),Path('Untitled30.ipynb'),Path('Untitled29.ipynb'),Path('Untitled3.ipynb'),Path('Untitled14.ipynb'),Path('Untitled4.ipynb'),Path('Untitled13.ipynb'),Path('Untitled6.ipynb'),Path('Untitled11.ipynb'),Path('Untitled1-Copy1.ipynb'),Path('Untitled15.ipynb'),Path('Untitled2.ipynb'),Path('Untitled28.ipynb'),Path('Untitled17.ipynb'),Path('Untitled26.ipynb'),Path('Untitled24.ipynb'),Path('Untitled19.ipynb')...]

In [94]:
new_paths = nbs.map(rename_notebook)

Renamed Untitled12.ipynb to 2025-01-11-Fastai-Tokenizers.ipynb
Renamed Untitled5.ipynb to 2025-01-06-Fastlite-With-Files.ipynb
Renamed Untitled1.ipynb to 2025-01-04-Tone-Js-And-FastHTML.ipynb
Renamed Untitled16.ipynb to 2025-01-10-Untitled.ipynb
Renamed Untitled30.ipynb to 2025-01-29-Untitled.ipynb
Renamed Untitled29.ipynb to 2025-01-26-Numberblocks-6.ipynb
Renamed Untitled3.ipynb to 2025-01-05-Fast-HTML-By-Example-Part-2.ipynb
Renamed Untitled14.ipynb to 2025-01-14-London-Kolkata-Manila-Brisbane-Time-Conversion.ipynb
Renamed Untitled4.ipynb to 2025-01-06-Updating-Git-Repos-With-Unstaged-Changes-Check.ipynb
Renamed Untitled13.ipynb to 2025-01-12-Numberblock-2.ipynb
Renamed Untitled6.ipynb to 2025-01-06-SQLite-CLI-Power-Users-Guide.ipynb
Renamed Untitled11.ipynb to 2025-01-16-Using-FastCaddy-With-MonsterUI.ipynb
Renamed Untitled1-Copy1.ipynb to 2025-01-04-Check-Each-Repo-For-Uncommitted-Changes.ipynb
Renamed Untitled15.ipynb to 2025-01-11-Untitled-Notebook.ipynb


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

Several of my notebooks were renamed successfully! I ran out of quota, though. I was probably hitting the Gemini API too fast. Let's see where we are and try again.

In [97]:
nbs = L(Path().glob("Untitled*.ipynb"))
nbs

(#16) [Path('Untitled2.ipynb'),Path('Untitled28.ipynb'),Path('Untitled17.ipynb'),Path('Untitled26.ipynb'),Path('Untitled24.ipynb'),Path('Untitled19.ipynb'),Path('Untitled20.ipynb'),Path('Untitled8.ipynb'),Path('Untitled22.ipynb'),Path('Untitled18.ipynb'),Path('Untitled25.ipynb'),Path('Untitled9-Copy1.ipynb'),Path('Untitled27.ipynb'),Path('Untitled23.ipynb'),Path('Untitled9.ipynb'),Path('Untitled21.ipynb')]

In [98]:
new_paths = nbs.map(rename_notebook)

Renamed Untitled2.ipynb to 2025-01-03-Fast-HTML-By-Example-Part-2.ipynb
Renamed Untitled28.ipynb to 2025-01-23-Fast-HTML-By-Example-Part-2.ipynb
Renamed Untitled26.ipynb to 2025-01-23-Using-fastlite-and-apswutils.ipynb
Renamed Untitled24.ipynb to 2025-01-19-Exploring-Lucide-Icons.ipynb
Renamed Untitled19.ipynb to 2025-01-14-Discord-Message-Time-Converter.ipynb
Renamed Untitled20.ipynb to 2025-01-16-Fast-HTML-By-Example-Part-2.ipynb
Renamed Untitled8.ipynb to 2025-01-07-Use-Pathlib-For-Paths-Not-Env-Vars.ipynb
Renamed Untitled22.ipynb to 2025-01-17-AAI-Meeting-Notes-2025-01-17.ipynb
Renamed Untitled18.ipynb to 2025-01-12-Untitled.ipynb
Renamed Untitled25.ipynb to 2025-01-21-Making-CLI-Tools-With-Fastcore-Script.ipynb
Renamed Untitled9-Copy1.ipynb to 2025-01-07-FtResponse-In-FastHTML.ipynb
Renamed Untitled27.ipynb to 2025-01-23-Modifying-Execnb-Render_outputs-To-Use-Monsterui.ipynb
Renamed Untitled23.ipynb to 2025-01-26-Understanding-FastHTMLs-FT-Class.ipynb
Renamed Untitled21.ipynb to 2

Mostly done! The 2 warnings sound like I have duplicates.

In [99]:
nbs = L(Path().glob("Untitled*.ipynb"))
nbs

(#2) [Path('Untitled17.ipynb'),Path('Untitled9.ipynb')]

Finally, I'm checking those remaining notebooks. It appears those are variations on the ones that exist. I can manually merge those.